In [2]:
%load_ext autoreload
%autoreload 2

from dataclasses import dataclass
from querychains import with_context, Context, OpenAiChatEngine, start_server, AnthropicEngine, QueryEngine
import json

def show(obj):
    print(json.dumps(obj, indent=2))

In [3]:
# Minimal context
with Context("MyContext") as c:
    pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uid": "2023-06-23T10:29:40-MyContext-ogwxvb",
  "start_time": "2023-06-23T10:29:40.621858",
  "end_time": "2023-06-23T10:29:40.621891"
}


In [3]:
# Context nesting
with Context("MyContext") as c:
    with Context("ChildContext1"):
        pass
    with Context("ChildContext2"):
        pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uuid": "2d712c36-64c4-4008-8cbc-bcd6860c78da",
  "children": [
    {
      "_type": "Context",
      "name": "ChildContext1",
      "uuid": "0e6533ac-b0eb-43cf-839e-594e895f8c4a",
      "start_time": "2023-06-21T15:19:41.875022",
      "end_time": "2023-06-21T15:19:41.875033"
    },
    {
      "_type": "Context",
      "name": "ChildContext2",
      "uuid": "686298ff-9494-4242-9a02-6ac3501e8ed2",
      "start_time": "2023-06-21T15:19:41.875045",
      "end_time": "2023-06-21T15:19:41.875052"
    }
  ],
  "start_time": "2023-06-21T15:19:41.874994",
  "end_time": "2023-06-21T15:19:41.875055"
}


In [4]:
# Inputs and outputs
with Context("MyContext", inputs={"a": 10, "b": 20}) as c:
    c.set_result(30)

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uid": "2023-06-23T10:30:04-MyContext-KxP1P0",
  "inputs": {
    "a": 10,
    "b": 20
  },
  "result": 30,
  "start_time": "2023-06-23T10:30:04.909438",
  "end_time": "2023-06-23T10:30:04.909461"
}


In [5]:
# Context decorator
@with_context
def my_function(a, b):
    return a + b

with Context("root") as c:
    my_function(10, 20)
    
show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uid": "2023-06-23T10:30:10-root-MmZDm3",
  "children": [
    {
      "_type": "Context",
      "name": "my_function",
      "uid": "2023-06-23T10:30:10-my_function-DaPZgE",
      "kind": "call",
      "inputs": {
        "a": 10,
        "b": 20
      },
      "result": 30,
      "start_time": "2023-06-23T10:30:10.771345",
      "end_time": "2023-06-23T10:30:10.771365"
    }
  ],
  "start_time": "2023-06-23T10:30:10.771278",
  "end_time": "2023-06-23T10:30:10.771372"
}


In [6]:
# Error handling

@with_context
def my_function(a, b):
    raise Exception("Oops")

try:
    with Context("root") as c:
        my_function(10, 20)
except:
    pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uid": "2023-06-23T10:30:29-root-V4kM9X",
  "state": "error",
  "error": {
    "_type": "error",
    "name": "Oops"
  },
  "children": [
    {
      "_type": "Context",
      "name": "my_function",
      "uid": "2023-06-23T10:30:29-my_function-OIsjcp",
      "state": "error",
      "kind": "call",
      "inputs": {
        "a": 10,
        "b": 20
      },
      "error": {
        "_type": "error",
        "name": "Oops"
      },
      "start_time": "2023-06-23T10:30:29.511410",
      "end_time": "2023-06-23T10:30:29.511442"
    }
  ],
  "start_time": "2023-06-23T10:30:29.511324",
  "end_time": "2023-06-23T10:30:29.511451"
}


In [7]:
# Dataclasses

@dataclass
class Person:
    name: str
    age: int
    
@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uid": "2023-06-23T10:30:48-root-yVes3H",
  "children": [
    {
      "_type": "Context",
      "name": "say_hi",
      "uid": "2023-06-23T10:30:48-say_hi-uk8iKQ",
      "kind": "call",
      "inputs": {
        "person": {
          "name": "Alice",
          "age": 21,
          "_type": "Person"
        }
      },
      "result": "Hi Alice!",
      "start_time": "2023-06-23T10:30:48.932135",
      "end_time": "2023-06-23T10:30:48.932175"
    }
  ],
  "start_time": "2023-06-23T10:30:48.931991",
  "end_time": "2023-06-23T10:30:48.932187"
}


In [5]:
# Data browser

server = start_server(port=5000)  # Writes an URL where the server is running

INFO:     Started server process [5982]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57122 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:57122 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:57122 - "GET /contexts/list HTTP/1.1" 200 OK


In [7]:
@dataclass
class Person:
    name: str
    age: int

@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

server.add_context(c)

INFO:     127.0.0.1:54892 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:54892 - "GET /contexts/uuid/9b662c82-ef02-4a62-8f84-26880c61f936 HTTP/1.1" 200 OK
INFO:     127.0.0.1:54892 - "GET /contexts/uuid/3a93613a-dab4-402b-97ec-2c72d2a2e939 HTTP/1.1" 200 OK


In [ ]:
# LLM queries

engine = OpenAiChatEngine()

with Context("root") as c:
        response = engine.query("Hi are you?")
        engine.query("Is the following text genereted by LLM?\n\n" + response)
show(c.to_dict())

In [ ]:
# async LLM queries
import asyncio 

engine1 = OpenAiChatEngine()
engine2 = AnthropicEngine()

@with_context
async def make_queries(engine: QueryEngine):
    response = await engine.aquery("Hi are you?")
    return await engine.aquery("Is this nice response?\n\n" + response)  

with Context("root") as c:
    q1 = make_queries(engine1)
    q2 = make_queries(engine2)

    await q1
    await q2

show(c.to_dict())